# Creating a database with tables

In [1]:
import sqlite3
import pandas as pd



con = sqlite3.connect("ads.db")
cursor = con.cursor()
cursor.execute("CREATE TABLE case_ads(AD_ID number(6) PRIMARY KEY, AD_TYPE char(1), CREATION_DATE date, CLIENT_ID number(3));")
cursor.execute("CREATE TABLE case_ad_statistics(AD_ID number(6) REFERENCES case_ads(AD_ID), EVENT_DATE date, DELIVERIES  int, IMPRESSIONS int, CLICKS int, SPEND int);")
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
con.close()


[('case_ads',), ('case_ad_statistics',)]


# Adding data from csvs

In [ ]:

df_1 = pd.read_csv('case_ads.csv', sep = ';')
df_2 = pd.read_csv('case_ad_statistics.csv', sep = ';')
con = sqlite3.connect("ads.db")
cursor = con.cursor()

df_1.to_sql('case_ads', con, if_exists='append', index = False)
df_2.to_sql('case_ad_statistics', con, if_exists='append', index = False)

con.close()

# SQL  Queries

In [13]:
# Load sql module

%load_ext sql

In [14]:
%sql sqlite:///ads.db 

In [19]:
%%sql 

SELECT * 
FROM case_ads 




 * sqlite:///ads.db
Done.


AD_ID,AD_TYPE,CREATION_DATE,CLIENT_ID
442486,Type C,25.10.18,408
429153,Type A,25.10.18,402
425498,Type C,27.10.18,409
440554,Type B,23.10.18,401
432689,Type B,27.10.18,402
427747,Type A,29.10.18,407
446766,Type A,31.10.18,402
140447,Type A,25.10.18,401
443025,Type A,24.10.18,401
445347,Type E,25.10.18,401


In [20]:
%%sql 

SELECT * 
FROM case_ad_statistics


 * sqlite:///ads.db
Done.


AD_ID,EVENT_DATE,DELIVERIES,IMPRESSIONS,CLICKS,SPEND
415050,01.11.18,0,1,0,0
445347,23.11.18,116,25,0,0
425498,01.11.18,208,62,0,0
417943,06.11.18,713,168,0,0
140447,06.11.18,30,11,0,0
446094,19.11.18,37042,1528,0,0
440554,20.11.18,1427,304,3,5
440554,24.11.18,2537,486,2,18
427747,22.11.18,14006,1459,9,54
427747,01.11.18,754,166,0,0


##### Number of ads in the database:

In [23]:
%%sql 

SELECT COUNT(AD_ID) AS noOfAds
FROM case_ads

 * sqlite:///ads.db
Done.


noOfAds
19


##### Number of clients

In [24]:
%%sql 

SELECT COUNT(DISTINCT CLIENT_ID) AS noOfClients
FROM case_ads

 * sqlite:///ads.db
Done.


noOfClients
10


# Joining the tables
##### The tables have been joined in a way that is client oriented, if a client wants to see their overall statistics then this information would provide a quick and easy insight toward the clients' ads. The joined table includes the total number of ads a client has, the average delieveries, impressions, clicks and spend over the total number of ads. Although there could be additional ways to display this information like showing the total statistics per client rather than their averages but I think this average would be a more important metric for clients with higer statistics. 

In [126]:
%%sql
SELECT CLIENT_ID, COUNT(DISTINCT t2.AD_ID) TOTAL_ADS, SUM(DELIVERIES)/COUNT( DISTINCT t2.AD_ID) AVG_DELIVERIES, SUM(IMPRESSIONS)/COUNT( DISTINCT t2.AD_ID) AVG_IMPRESSIONS, SUM(CLICKS)/COUNT( DISTINCT t2.AD_ID) AVG_CLICKS, SUM(SPEND)/COUNT( DISTINCT t2.AD_ID) AVG_SPEND
FROM case_ads t1  LEFT JOIN case_ad_statistics t2
ON t1.AD_ID = t2.AD_ID
GROUP BY CLIENT_ID

 * sqlite:///ads.db
Done.


CLIENT_ID,TOTAL_ADS,AVG_DELIVERIES,AVG_IMPRESSIONS,AVG_CLICKS,AVG_SPEND
401,4,4573,1150,2,25
402,6,411,120,0,0
403,2,10070,693,3,9
404,1,2491,496,0,0
405,1,725,168,0,0
406,1,37042,1528,0,0
407,1,28312,4043,15,106
408,1,456,59,0,0
409,1,208,62,0,0
410,1,0,1,0,0


#### Spend per ad type

In [128]:
%%sql 

SELECT AD_TYPE, SUM(t2.SPEND) SPEND_AGGREGATE
FROM case_ads t1 
JOIN case_ad_statistics t2 
ON 
t1.AD_ID = t2.AD_ID
GROUP BY AD_TYPE 

 * sqlite:///ads.db
Done.


AD_TYPE,SPEND_AGGREGATE
Type A,124
Type B,101
Type C,0
Type D,2
Type E,0
